In [1]:
%%capture
%pip install -U langchain langchainhub langchain_community langchain-huggingface faiss-gpu transformers accelerate datasets

In [2]:
from langchain.document_loaders import HuggingFaceDatasetLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from transformers import AutoTokenizer, AutoModelForCausalLM,pipeline
from langchain_huggingface import HuggingFacePipeline
from langchain.chains import RetrievalQA

In [3]:
# prompt: huggingface login

!huggingface-cli login



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in 

In [7]:
# Specify the dataset name
dataset_name = "meta-math/MetaMathQA"

# Create a loader instance using dataset columns, specifying the correct column name
loader_doctor = HuggingFaceDatasetLoader(
    dataset_name,
    page_content_column="response"  # Change 'text' to 'problem'
)

# Load the data
doctor_data = loader_doctor.load()

# Select the first 1000 entries
doctor_data = doctor_data[:1000]

doctor_data[:2]

[Document(metadata={'type': 'MATH_AnsAug', 'query': "Gracie and Joe are choosing numbers on the complex plane. Joe chooses the point $1+2i$. Gracie chooses $-1+i$. How far apart are Gracie and Joe's points?", 'original_question': "Gracie and Joe are choosing numbers on the complex plane. Joe chooses the point $1+2i$. Gracie chooses $-1+i$. How far apart are Gracie and Joe's points?"}, page_content='"The distance between two points $(x_1,y_1)$ and $(x_2,y_2)$ in the complex plane is given by the formula $\\\\sqrt{(x_2-x_1)^2+(y_2-y_1)^2}$.\\nIn this case, Joe\'s point is $(1,2)$ and Gracie\'s point is $(-1,1)$.\\nSo the distance between their points is $\\\\sqrt{((-1)-(1))^2+((1)-(2))^2}=\\\\sqrt{(-2)^2+(-1)^2}=\\\\sqrt{4+1}=\\\\sqrt{5}$.\\nTherefore, Gracie and Joe\'s points are $\\\\boxed{\\\\sqrt{5}}$ units apart.\\nThe answer is: \\\\sqrt{5}"'),
 Document(metadata={'type': 'GSM_Rephrased', 'query': 'What is the total cost of purchasing equipment for all sixteen players on the footba

In [8]:
# # Specify the dataset name
# dataset_name = "meta-math/MetaMathQA"


# # Create a loader instance using dataset columns
# loader_doctor = HuggingFaceDatasetLoader(dataset_name)

# # Load the data
# doctor_data = loader_doctor.load()

# # Select the first 1000 entries
# doctor_data = doctor_data[:1000]

# doctor_data[:2]

In [6]:
# doctor_data

In [9]:
# Define the path to the embedding model
modelPath = "sentence-transformers/all-MiniLM-L12-v2"

# GPU acceleration
model_kwargs = {'device':'cuda'}

# Create a dictionary with encoding options
encode_kwargs = {'normalize_embeddings': False}

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)
text = "Convert $10101_3$ to a base 10 integer"
query_result = embeddings.embed_query(text)
query_result[:3]

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:151: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

[-0.032555919140577316, 0.0056881834752857685, -0.030486004427075386]

In [ ]:
# /content/sample_data

In [10]:
vector_db = FAISS.from_documents(doctor_data, embeddings)
vector_db.save_local("/content/faiss_doctor_index")
question = "Convert $10101_3$ to a base 10 integer."
searchDocs = vector_db.similarity_search(question)
print(searchDocs[0].page_content)

"$10101_3 = 1 \\cdot 3^4 + 0 \\cdot 3^3 + 1 \\cdot 3^2 + 0 \\cdot 3^1 + 1 \\cdot 3^0 = 81 + 9 + 1 = \\boxed{91}$.\nThe answer is: 91"


In [10]:
'''
Hello. You have a persistent headache in the frontal part of your head.
Was there fever, chills, and rigor, were you feeling like vomiting, after taking those drugs prescribed for you?
Did your headache subside? The medicines given at the pharmacy are fine for malaria, but it may not have been malaria,
but it could be Salmonella typhi infection, which causes typhoid fever. Worm medicine is fine in the case. I will advise you to go to a lab
and do blood film for malaria parasite, do Widal test and stool microscopy culture and sensitivity. If there is salmonella infection E coli
or any enterobacteria infection, the stool MCS will detect it. If there is worm infestation such as hookworm or tapeworm,
it will also pick it in microscopy. You can take tablet Buscopan (hyoscine butylbromide) 10 mg thrice daily.
Then get tablet Ciprofloxacin 500 mg bd (twice daily) take them for five days. You can add tablet Rabeprazole 20 mg bd in case if
there is any peptic ulcer since the description of the pain and the location in your abdomen was not very precise. Enteric fever.
Malaria. Intestinal spasm. PUD(peptic ulcer disease)"

'''

'\nHello. You have a persistent headache in the frontal part of your head. \nWas there fever, chills, and rigor, were you feeling like vomiting, after taking those drugs prescribed for you? \nDid your headache subside? The medicines given at the pharmacy are fine for malaria, but it may not have been malaria, \nbut it could be Salmonella typhi infection, which causes typhoid fever. Worm medicine is fine in the case. I will advise you to go to a lab \nand do blood film for malaria parasite, do Widal test and stool microscopy culture and sensitivity. If there is salmonella infection E coli \nor any enterobacteria infection, the stool MCS will detect it. If there is worm infestation such as hookworm or tapeworm, \nit will also pick it in microscopy. You can take tablet Buscopan (hyoscine butylbromide) 10 mg thrice daily. \nThen get tablet Ciprofloxacin 500 mg bd (twice daily) take them for five days. You can add tablet Rabeprazole 20 mg bd in case if \nthere is any peptic ulcer since the 

In [11]:
retriever = vector_db.as_retriever()

In [12]:
question = "What is the total cost of purchasing equipment for all sixteen  players on the football team, considering that each player requires a $25 jersey, a $15.20 pair of shorts, and a pair of socks priced at $6.80? "
searchDocs = vector_db.similarity_search(question)
searchDocs[0].page_content

'"Each player requires a $25 jersey, a $15.20 pair of shorts, and a pair of socks priced at $6.80.\\nSo the total cost for each player is $25 + $15.20 + $6.80 = $47.\\nSince there are sixteen players on the football team, the total cost for all of them is 16 * $47 = $752.\\n#### 752\\nThe answer is: 752"'

In [13]:
question = "A rectangular garden has a length that is 5 meters longer than its width. If the perimeter of the garden is 50 meters, what are the dimensions of the garden."
searchDocs = vector_db.similarity_search(question)
searchDocs[0].page_content

'"The perimeter of the original garden is $40+10+40+10=100$ feet.\\nTo form a square with the same perimeter, each side length must be $100/4=25$ feet.\\nThe area of the original garden is $40\\\\cdot 10=400$ square feet, and the area of the new garden is $25\\\\cdot 25=625$ square feet.\\nThe new garden is $625-400=\\\\boxed{225}$ square feet larger than the old garden.\\nThe answer is: 225"'

In [14]:
import torch
base_model = "openchat/openchat-3.6-8b-20240522"

tokenizer = AutoTokenizer.from_pretrained(base_model)

model = AutoModelForCausalLM.from_pretrained(
        base_model,
        return_dict=True,
        low_cpu_mem_usage=True,
        torch_dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True,
)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=120
)

llm = HuggingFacePipeline(pipeline=pipe)

tokenizer_config.json:   0%|          | 0.00/51.2k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/712 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [15]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


# rag_prompt = hub.pull("rlm/rag-prompt")



In [16]:
rag_prompt = hub.pull("rlm/rag-prompt", api_key="YOUR_API_KEY")


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


In [18]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.vectorstores import Chroma # Assuming you're using Chroma

# ... (your existing code) ...

# Assuming 'vector_db' is your Chroma instance
retriever = vector_db.as_retriever()

qa_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)

In [19]:
# qa_chain = (
#     {"context": retriever, "question": RunnablePassthrough()}
#     | rag_prompt
#     | llm
#     | StrOutputParser()
# )

In [20]:
question = "A rectangular garden has a length that is 5 meters longer than its width. If the perimeter of the garden is 50 meters, what are the dimensions of the garden"
result = qa_chain.invoke(question)
print(result.split("Answer: ")[1])

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


The perimeter of the garden is 50 meters, so the width plus the length is 25 meters. Let the width be w, then the length is w + 5. The perimeter is 2w + 2(w + 5) = 50, which simplifies to 4w + 10 = 50, which simplifies to 4w = 40, which simplifies to w = 10. The length is 10 + 5 = 15. The dimensions of the garden are 10 meters by 15 meters.10 meters by


In [23]:
# You have a headache. You should take tablet Buscopan (hyoscine butylbromide) 10 mg thrice daily.
# Then get tablet Ciprofloxacin 500 mg bd (twice daily) take them for five days.
# You can add tablet Rabeprazole 20 mg bd in case if there is any peptic ulcer since
# the description of the pain and the location in your abdomen was not very precise.:
# You have a headache. You should take tablet Buscopan (hyoscine butylbromide) 10



In [21]:
question = "In a right triangle, the angle 𝜃  has a tangent value of 3/4. If the length of the adjacent side to θ is 8 units, find the length of the opposite side and the hypotenuse of the triangle"
result = qa_chain.invoke(question)
print(result.split("Answer: ")[1])

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


The length of the opposite side is 10 units and the length of the hypotenuse is 14 units.
Since 𝜃 is an acute angle in a right triangle, we can use the tangent function to find the length of the opposite side. The tangent of an angle is equal to the ratio of the length of the opposite side to the length of the adjacent side. In this case, we have tan(𝜃) = 3/4, so the length of the opposite side is (3/4) * 8 = 6 units. To


In [ ]:
# To stop nightfall, you can try taking Fenugreek with honey daily, eating curd,
# drinking gourd juice, eating onions, taking relaxing baths, relaxing your mind,
# avoiding tea, coffee, and alcohol,
# eating healthy foods, drinking plenty of water, exercising, and practicing yoga.
# I am suffering from nightfall for past three months.
# Please give solution to stop nightfall. Practice the following to control the nightfall.
# 1) Take Fenugreek (methi) with honey daily. 2) Take curd daily as it helps to